# Web Scraping Intro

### Hypertext Transfer Protocol (HTTP) is the foundation for data communication on the world wide web
- Entering a URL is a request for the resource at that domain address
- Response is what happens (page loads? 404 error?)

To retrieve the contents of a website, we will be using the [_requests_](https://requests.readthedocs.io/en/master/) library.

In [ ]:
import requests

In this notebook, we will be using a **GET** request. This is a request for data from a specified resource.  

Another common type or request is a **POST** request. POST submits data to be processed (e.g., from an HTML form) to the identified resource. The data is included in the body of the request. This may result in the creation of a new resource or the updates of existing resources or both.

To perform a GET request, use `requests.get()` and pass in the desired url.

In [ ]:
URL = 'http://en.wikipedia.org/wiki/Turing_Award'

response = requests.get(URL)

Let's see what kind of object we get.

In [ ]:
type(response)

We can check the status code using the `status_code` attribute.

In [ ]:
response.status_code

A 200 status code is the standard response for a successful request.  

Other common status codes:
 * 400: Bad Request
 * 404: Not Found

Let's see what happens if we request a non-existent URL.

In [ ]:
requests.get('https://en.wikipedia.org/wiki/Tuning_Award')

Back to the good correct request, let's see what this request returned.

In [ ]:
response.text

It is very hard to decipher the above text. Luckily for us, the [_Beautiful Soup_](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) library comes to the rescue. This library assists us in parsing HTML into something usable.

In [ ]:
from bs4 import BeautifulSoup as BS

First, we can soupify our response text. Since we are working with HTML, we can specify that we need the html parser.

In [ ]:
soup = BS(response.text)

Now, we can print it out in a slightly more readable form.

In [ ]:
print(soup.prettify())

Beautiful Soup lets us search through this HTML and extract out the contents we want by tag.  

Say we wanted to find the title of this page. We can accomplish this by using the `.find` method on our soup and then prining its `.text` attribute.

In [ ]:
soup.find('title')

To extract out the text, you can use the `.text` attribute.

In [ ]:
soup.find('title').text

We can find all elements with a particular tag using the `.findAll(<tag>)` method. 

In [ ]:
images = soup.findAll('img')
print(type(images))
images

Let's look closer at the first image.

In [ ]:
first_image = images[0]
print(type(first_image))
first_image

You can access attributes of a Tag object in the same way that you would access values from a dictionary.

In [ ]:
first_image['src']

You can also safely access attributes using `.get`. This might be useful if, for example, you aren't sure if a particular Tag or all tags had a particular attribute.

In [ ]:
# Non-safe
first_image['class']

In [ ]:
# Safe
first_image.get('class')

In [ ]:
first_image.get('class', default = 'No Class')

If you want to grab a particular attribute for all images, an easy way to do so is with a list comprehension.

In [ ]:
image_srcs = [x.get('src') for x in images]

In [ ]:
image_srcs

We can further navigate the html tree to extract out other bits of information.

When scraping from a web page, you should make use of "View Page Source" and/or "Inspect Element" in your web browswer.

For example, let's say we want to look at the third div on the page.

In [ ]:
soup.findAll('div')[2]

Similar to using `find` and `findall` in the full soup, we can use the `.find` method just within a Tag.

In [ ]:
soup.findAll('div')[2].find('h1')

In [ ]:
soup.findAll('div')[2].find('h1')['id']

In [ ]:
soup.findAll('div')[2].find('h1').text

Now, let's look for the table containing the Turing Award winners.

Using `.findAll` reveals that there are multiple tables on the page.

In [ ]:
soup.findAll('table')

If we know a bit more about what we are looking for, we can include an `attrs` argument and pass a dictionary. 

Go to the Turing award page in your browser, right click on the top of the table and choose "Inspect". You will notice that this table is defined with tag `<table class="wikitable">.` Armed with this information, we can narrow down our search.

In [ ]:
soup.find('table', attrs={'class' : 'wikitable'})

We can display the table by importing the `HTML` function.

In [ ]:
table_html = str(soup.find('table', attrs={'class' : 'wikitable'}))

from IPython.core.display import HTML

HTML(table_html)

However, this does not give us a way to work with the data in the table, only to display it.

As part of Data Question 3, your group will need to figure out how to convert the resulting table into a `pandas` DataFrame.

## Using the USGS API

The USGS provides an API for retrieving data about earthquakes. This API is accessible at https://earthquake.usgs.gov/fdsnws/event/1.

To use this API, one option is to simply use your browser - modify the query string to find what you're looking for. For example, to fetch all earthquakes that occurred between Jan. 1, 2019 and Jan. 2, 2019, you can use:
https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=2019-01-01&endtime=2019-01-02

However, this method is kind of clunky. We can also use the requests library to help us. This time, instead of doing just a GET request on a particular url, we will pass a payload containing our desired parameters as a dictionary.   

Let's say we are interested in finding all earthquakes that occurred since January 1 that were at least magnitude 6.0.

In [ ]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'

We can create a dictionary of parameters that will get passed in with our request.

In [ ]:
payload = {'format': 'csv', 
           'starttime': '2020-01-01',
           'minmagnitude' : '6.0'}

In [ ]:
r = requests.get(url=url, params=payload)

If you want to inspect the resulting url, you can access the response url attribute.

In [ ]:
print(r.url)

Let' see what is returned:

In [ ]:
print(r.text[:1000])

There are a few ways we can now proceed if we want to convert this response into a DataFrame. First, we can read the text into a DataFrame using `read_csv`. To get this to work, we have to first pass the text through the StringIO method.

In [ ]:
import pandas as pd
from io import StringIO

In [ ]:
eq = pd.read_csv(StringIO(r.text))
eq.head()

Another way to make it work is to pass the url itself to `read_csv`.

In [ ]:
eq = pd.read_csv(r.url)
eq.head()

Finally, we can save the text as a csv file and then read it back in using pandas:

In [ ]:
with open('eq.csv', 'w') as fi:
    fi.write(r.text)

In [ ]:
eq = pd.read_csv('eq.csv')
eq.head()